In [1]:
from SeqEN2.utils.data_loader import read_json, write_json
from pathlib import Path
from IPython.display import clear_output
from glob import glob
from os import system

In [48]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
import prody

In [4]:
new_wd = Path('../../data/pdb_dssp_phipsi_ndxed/')
if not new_wd.exists():
    new_wd.mkdir()
pdb_files = Path('../../data/pdb_files/')
if not pdb_files.exists():
    pdb_files.mkdir()

In [79]:
data_dir = '../../data/pdb_dssp_phipsi/'
files = sorted(glob(f'{data_dir}*.json.gz'))

aa_dict = {
    "A": "ALA", "C": "CYS", "D": "ASP", "E": "GLU", "F": "PHE", "G": "GLY", "H": "HIS", "I": "ILE", "K": "LYS", "L": "LEU", 
    "M": "MET", "N": "ASN", "P": "PRO", "Q": "GLN", "R": "ARG", "S": "SER", "T": "THR", "V": "VAL", "W": "TRP", "Y": "TYR"
}
aa_dict_r = {item:key for key, item in aa_dict.items()}
aa_dict_r['KCX'] = 'K'
aa_dict_r['MSE'] = 'M'

In [89]:
def get_psiphi(d, aa, i, diff): 
    try:
        return d[f"{aa_dict[aa]}-{i+diff}"]
    except KeyError:
        if aa == 'K':
            try:
                return d[f"KCX-{i+diff}"]
            except:
                return [np.nan, np.nan]
        elif aa == 'M':
            try:
                return d[f"MSE-{i+diff}"]
            except:
                return [np.nan, np.nan]
        return [np.nan, np.nan]
    
def diff(d, seq, p=False):
    keys = sorted([(int(k.split('-')[1]), aa_dict_r[k.split('-')[0]]) for k in d.keys()])[:10]
    if p:
        print(keys)
    delta = keys[0][0] - seq.find(''.join([item[1] for item in keys]))
    if p:
        print(seq.find(''.join([item[1] for item in keys])))
        print(delta)
    return delta
    
dropped_seq = {}
seqs = []
for file in tqdm(files[:]):
    # print(f'transforming {file}')
    filename = file.split('/')[-1]
    data = read_json(file)
    keys = list(data.keys())
    new_data = {}
    aa_keys = "WYFMILVAGPSTCEDQNHRK*"
    ss_keys = "LSTIGHBE*"
    w = 20
    padding = w - 1
    for key in keys:
        item = data[key]
        if len(item["Sequence"]) < 40:
            dropped_seq[key] = {"item": item, "error": "shorter than 40 aa"}
            continue
        if item["PsiPhi"] == {}:
            dropped_seq[key] = {"item": item, "error": "empty psiphi"}
            continue
        seq = ("*" * padding) + item["Sequence"] + ("*" * padding)
        if seq not in seqs:
            seqs.append(seq)
            ss_seq = ("*" * padding) + item["DSSP"] + ("*" * padding)
            assert len(seq) == len(ss_seq), f"{key} failed to transform, seq and ss have different lengths."
            try:
                delta = diff(item["PsiPhi"], seq)
                values = np.array([get_psiphi(item["PsiPhi"], aa, i, delta) for i, aa in enumerate(seq)]).reshape((-1, 2))
                if np.sum(np.isnan(new_form['psi'][19:-19])) > 0.9*len(item["PsiPhi"]):
                    pass
                    # print(key)
                    # # print(new_form['psi'][19:-19])
                    # print(np.sum(np.isnan(new_form['psi'][19:-19])))
                    # print(0.8*len(item["PsiPhi"]))
                    # print(item)
                    # print(new_form)
                    dropped_seq[key] = {"item": item, "error": "ndx mismatch"}
                    continue
                new_form = {
                    'ndx': np.array([aa_keys.index(aa) for aa in seq], dtype=np.int8),
                    'ss': np.array([ss_keys.index(ss) for ss in ss_seq], dtype=np.int8),
                    'psi': values[:, 0],
                    'phi': values[:, 1]
                }
                new_data[key] = new_form
            except ValueError:
                dropped_seq[key] = {"item": item, "error": "non-cononical aa"}
                continue
            except KeyError:
                dropped_seq[key] = {"item": item, "error": "non-cononical aa"}
                continue
            # except IndexError:
            #     print('Inddex errrrrrrrro')
            #     print(item)
            #     _ = diff(item["PsiPhi"], seq, p=True)
            #     break
        else:
            dropped_seq[key] = {"item": item, "error": "repeating"}
    write_json(new_data, str(new_wd / filename), encoder='numpy')
    
print(dropped_seq.keys())

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 159/159 [04:48<00:00,  1.81s/it]

dict_keys(['103M_1_A', '105M_1_A', '107M_1_A', '108M_1_A', '208L_1_A', '209D_2_C', '20GS_1_A', '110M_1_A', '111M_1_A', '112M_1_A', '316D_2_C', '11BG_1_A', '222L_1_A', '225L_1_A', '226L_1_A', '228L_1_A', '12AS_1_A', '12GS_1_A', '132L_1_A', '43CA_1_A', '13GS_1_A', '14GS_1_A', '451C_1_A', '154L_1_A', '155C_1_A', '256L_1_A', '456C_1_A', '258L_1_A', '259L_1_A', '25C8_2_H', '35C8_1_L', '35C8_2_H', '160L_1_A', '260L_1_A', '161L_1_A', '261L_1_A', '162L_1_A', '262L_1_A', '163L_1_A', '164L_1_A', '165L_1_A', '166L_1_A', '966C_1_A', '167L_1_A', '168L_1_A', '169L_1_A', '16GS_1_A', '16PK_1_A', '173D_2_C', '178L_1_A', '180L_1_A', '182L_1_A', '183L_1_A', '184L_1_A', '484D_2_A', '185D_1_A', '185L_1_A', '186L_1_A', '187L_1_A', '188L_1_A', '18GS_1_A', '1914_1_A', '193L_1_A', '194L_1_A', '19GS_1_A', '1A01_1_A', '1A02_3_N', '1A02_4_F', '1A02_5_J', '1A07_1_A', '1A08_1_A', '1A09_1_A', '1A0A_3_A', '1A0B_1_A', '1A0H_1_A', '1A0H_2_B', '1A0K_1_A', '1A0M_1_A', '1A0N_1_A', '1A0N_2_B', '1A0O_1_A', '1A0O_2_B', '1A0R

In [ ]:
values[0, 0] = 1.0
values

In [90]:
len(dropped_seq.keys())

156954

In [61]:
print(len(dropped_seq.keys()))

11205


In [62]:
try_again = [key for key, item in dropped_seq.items() if item['error'] == "non-cononical aa"]

In [63]:
len(try_again)

1309

In [64]:
dropped_seq_2 = {}
aa_keys = "WYFMILVAGPSTCEDQNHRK*"
ss_keys = "LSTIGHBE*"
w = 20
padding = w - 1
new_data = {}
for key in try_again:
    seq = dropped_seq[key]['item']['Sequence']
    striped_seq = seq.strip('X')
    if 'X' not in striped_seq:
        leading_x = len(seq) - len(seq.lstrip('X'))
        leading_x = None if leading_x == 0 else leading_x
        trailing_x = len(seq) - len(seq.rstrip('X'))
        trailing_x = None if trailing_x == 0 else -trailing_x
        if leading_x or trailing_x:
            striped_ss = dropped_seq[key]['item']['DSSP'][leading_x:trailing_x]
            if len(striped_seq) == len(striped_ss):
                seq = ("*" * padding) + striped_seq + ("*" * padding)
                ss_seq = ("*" * padding) + striped_ss + ("*" * padding)
                try:
                    values = np.array([aa_keys.index(aa) for aa in seq], dtype=np.int8).reshape((-1, 1))
                    values = np.append(values, np.array([ss_keys.index(ss) for ss in ss_seq], dtype=np.int8).reshape((-1, 1)), axis=1)
                    values = np.append(values, np.array([get_psiphi(item["PsiPhi"], aa, i-padding) for i, aa in enumerate(seq)]).reshape((-1, 2)), axis=1)
                    new_data[key] = values
                except ValueError:
                    dropped_seq_2[key] = {"item": dropped_seq[key]['item'], "error": "non-cononical aa"}
                    continue
            else:
                dropped_seq_2[key] = {"item": dropped_seq[key]['item'], "error": "trim not working"}
        
    else:
        dropped_seq_2[key] = {"item": dropped_seq[key]['item'], "error": "internal X"}
write_json(new_data, str(new_wd / "leading_trailing_X_fixed.json.gz"), encoder='numpy')

In [65]:
print(len(dropped_seq_2.keys()))

392


In [66]:
try_again = [key for key, item in dropped_seq_2.items() if item['error'] == "internal X"]

In [67]:
dropped_seq_3 = {}
aa_keys = "WYFMILVAGPSTCEDQNHRK*"
ss_keys = "LSTIGHBE*"
w = 20
padding = w - 1
new_data = {}
for key in try_again:
    seq = dropped_seq_2[key]['item']['Sequence']
    striped_seq = seq.strip('X')
    if len(striped_seq) - len(striped_seq.replace('X', '')) == 1:
        striped_seq = seq.strip('X').replace('X', 'A')
        leading_x = len(seq) - len(seq.lstrip('X'))
        leading_x = None if leading_x == 0 else leading_x
        trailing_x = len(seq) - len(seq.rstrip('X'))
        trailing_x = None if trailing_x == 0 else -trailing_x
        if leading_x or trailing_x:
            striped_ss = dropped_seq_2[key]['item']['DSSP'][leading_x:trailing_x]
            if len(striped_seq) == len(striped_ss):
                seq = ("*" * padding) + striped_seq + ("*" * padding)
                ss_seq = ("*" * padding) + striped_ss + ("*" * padding)
                try:
                    values = np.array([aa_keys.index(aa) for aa in seq], dtype=np.int8).reshape((-1, 1))
                    values = np.append(values, np.array([ss_keys.index(ss) for ss in ss_seq], dtype=np.int8).reshape((-1, 1)), axis=1)
                    values = np.append(values, np.array([get_psiphi(item["PsiPhi"], aa, i-padding) for i, aa in enumerate(seq)]).reshape((-1, 2)), axis=1)
                    new_data[key] = values
                except ValueError:
                    dropped_seq_3[key] = {"item": dropped_seq_2[key]['item'], "error": "non-cononical aa"}
                    continue
        
    else:
        dropped_seq_3[key] = {"item": dropped_seq_2[key]['item'], "error": "internal X more than 1"}
write_json(new_data, str(new_wd / "fixed_with_one_internal_X.json.gz"), encoder='numpy')

In [71]:
print(len(dropped_seq_3.keys()))

113


In [73]:
try_again = [key for key, item in dropped_seq_3.items() if item['error'] == "internal X more than 1"]

113

In [74]:
dropped_seq_4 = {}
aa_keys = "WYFMILVAGPSTCEDQNHRK*"
ss_keys = "LSTIGHBE*"
w = 20
padding = w - 1
new_data = {}
for key in try_again:
    seq = dropped_seq_3[key]['item']['Sequence']
    striped_seq = seq.strip('X')
    if len(striped_seq) - len(striped_seq.replace('X', '')) == 2:
        striped_seq = seq.strip('X').replace('X', 'A')
        leading_x = len(seq) - len(seq.lstrip('X'))
        leading_x = None if leading_x == 0 else leading_x
        trailing_x = len(seq) - len(seq.rstrip('X'))
        trailing_x = None if trailing_x == 0 else -trailing_x
        if leading_x or trailing_x:
            striped_ss = dropped_seq_3[key]['item']['DSSP'][leading_x:trailing_x]
            if len(striped_seq) == len(striped_ss):
                seq = ("*" * padding) + striped_seq + ("*" * padding)
                ss_seq = ("*" * padding) + striped_ss + ("*" * padding)
                try:
                    values = np.array([aa_keys.index(aa) for aa in seq], dtype=np.int8).reshape((-1, 1))
                    values = np.append(values, np.array([ss_keys.index(ss) for ss in ss_seq], dtype=np.int8).reshape((-1, 1)), axis=1)
                    values = np.append(values, np.array([get_psiphi(item["PsiPhi"], aa, i-padding) for i, aa in enumerate(seq)]).reshape((-1, 2)), axis=1)
                    new_data[key] = values
                except ValueError:
                    dropped_seq_4[key] = {"item": dropped_seq_3[key]['item'], "error": "non-cononical aa"}
                    continue
        
    else:
        dropped_seq_4[key] = {"item": dropped_seq_3[key]['item'], "error": "internal X more than 2"}
write_json(new_data, str(new_wd / "fixed_with_two_internal_X.json.gz"), encoder='numpy')

In [75]:
print(len(dropped_seq_4.keys()))

94


In [76]:
try_again = [key for key, item in dropped_seq_4.items() if item['error'] == "internal X more than 2"]
len(try_again)

94

In [77]:
dropped_seq_5 = {}
aa_keys = "WYFMILVAGPSTCEDQNHRK*"
ss_keys = "LSTIGHBE*"
w = 20
padding = w - 1
new_data = {}
for key in try_again:
    seq = dropped_seq_4[key]['item']['Sequence']
    striped_seq = seq.strip('X')
    if len(striped_seq) - len(striped_seq.replace('X', '')) == 3:
        striped_seq = seq.strip('X').replace('X', 'A')
        leading_x = len(seq) - len(seq.lstrip('X'))
        leading_x = None if leading_x == 0 else leading_x
        trailing_x = len(seq) - len(seq.rstrip('X'))
        trailing_x = None if trailing_x == 0 else -trailing_x
        if leading_x or trailing_x:
            striped_ss = dropped_seq_4[key]['item']['DSSP'][leading_x:trailing_x]
            if len(striped_seq) == len(striped_ss):
                seq = ("*" * padding) + striped_seq + ("*" * padding)
                ss_seq = ("*" * padding) + striped_ss + ("*" * padding)
                try:
                    values = np.array([aa_keys.index(aa) for aa in seq], dtype=np.int8).reshape((-1, 1))
                    values = np.append(values, np.array([ss_keys.index(ss) for ss in ss_seq], dtype=np.int8).reshape((-1, 1)), axis=1)
                    values = np.append(values, np.array([get_psiphi(item["PsiPhi"], aa, i-padding) for i, aa in enumerate(seq)]).reshape((-1, 2)), axis=1)
                    new_data[key] = values
                except ValueError:
                    dropped_seq_5[key] = {"item": dropped_seq_4[key]['item'], "error": "non-cononical aa"}
                    continue
        
    else:
        dropped_seq_5[key] = {"item": dropped_seq_4[key]['item'], "error": "internal X more than 3"}
write_json(new_data, str(new_wd / "fixed_with_three_internal_X.json.gz"), encoder='numpy')

In [78]:
print(len(dropped_seq_5.keys()))

89


In [79]:
try_again = [key for key, item in dropped_seq_5.items() if item['error'] == "internal X more than 3"]
len(try_again)

89

In [10]:
# read_json('../../data/pdb_dssp_phipsi/pdb_dssp_phipsi_0.json.gz')

In [30]:
def get_psiphi(d, aa, i): 
    try:
        return d[f"{aa_dict[aa]}-{i+1}"]
    except KeyError:
        return [np.nan, np.nan]
    
dropped_seq = {}
dropped_seq_domain = {}
        
for file in files[:1]:
    print(f'transforming {file}')
    filename = file.split('/')[-1]
    data = read_json(file)
    keys = list(data.keys())
    new_data = {}
    pfam_domains = {}
    aa_keys = "WYFMILVAGPSTCEDQNHRK*"
    ss_keys = "LSTIGHBE*"
    w = 20
    padding = w - 1
    i = 0
    for key in keys:
        item = data[key]
        pdbID = key.split('_')[0]
        try:
            pfam_domains[key] = prody.searchPfam(pdbID)
            pdb_file = f'{pdbID}.pdb.gz'
            system(f'mv {pdb_file} {pdb_files/pdb_file}')
        except:
            dropped_seq_domain[key] = "pfam failed"
        break
#         if len(item["Sequence"]) < 50:
#             dropped_seq[key] = {"item": item, "error": "shorter than 50 aa"}
#             continue
#         seq = ("*" * padding) + item["Sequence"] + ("*" * padding)
#         ss_seq = ("*" * padding) + item["DSSP"] + ("*" * padding)
#         assert len(seq) == len(ss_seq), f"{key} failed to transform, seq and ss have different lengths."
#         length = len(seq)
#         try:
#             values = np.array([aa_keys.index(aa) for aa in seq], dtype=np.int8).reshape((-1, 1))
#             values = np.append(values, np.array([ss_keys.index(ss) for ss in ss_seq], dtype=np.int8).reshape((-1, 1)), axis=1)
#             values = np.append(values, np.array([get_psiphi(item["PsiPhi"], aa, i-padding) for i, aa in enumerate(seq)]).reshape((-1, 2)), axis=1)
#             new_data[key] = values
#         except ValueError:
#             dropped_seq[key] = {"item": item, "error": "non-cononical aa"}
#             continue
        

        # store domain names
        
        ##
        
        
    write_json(new_data, str(new_wd / filename), encoder='numpy')

{'200L_1_A': 'pfam failed'}